In [2]:
from IPython.display import HTML
from IPython.display import display, Image
import casadi as ca
from casadi import jacobian
from casadi import gradient
from casadi import Function
from casadi import vertcat

<h1 align="center" style="font-size: 35px;"><font color="C1772D"><strong>Aplicación del Control Óptimo en el Campo de la Robótica</strong></font></h1>

<h2 align="center"><font color="#667388"><strong>Luis F. Recalde</strong><br>Ambato-Ecuador</font></h2>


<h3 align="center"><font color="#667388"><strong>Universidad Indoamerica</strong></font></h3>

<h3 style="font-size: 30px; color: #C1772D; font-weight: bold; text-align: left;">Optimización</h3>


<h3 align="left" style="font-size: 20px; color: #667388;"><strong>Notación</strong></h3>


<div style="text-align: justify; font-size: 20px;"> En general, de los conceptos que se van a tratar existen diferentes notaciones, especialmente de dos áreas: "Machine Learning" y "Optimization and Control". Si no se definen de forma adecuada, pueden llegar a provocar problemas durante la codificación de los algoritmos. </div>



<div style="text-align: justify; font-size: 20px;"> Se presentará una notación que es fácil de entender y que funciona para todos los casos. Cabe mencionar que quizás no todos estén de acuerdo con esta nomenclatura.</div>


<div style="text-align: justify; font-size: 20px;"> Considerar la siguiente funcion:</div>


<span style="font-size:1.4em;">
\begin{equation}
   {f}(\mathbf{x}): \mathbb{R}^{n} \rightarrow \mathbb{R} 
\end{equation}
</span>

<div style="text-align: justify; font-size: 20px;"> De la cual se puede calcular su derivada parcial de la siguiente forma:</div>


<span style="font-size:1.4em;">
\begin{equation}
   \frac{\partial {f}}{\partial \mathbf{x}} \in \mathbb{R}^{1 \times n} 
\end{equation}
</span>

<div style="text-align: justify; font-size: 20px;">el cual siempre es un vector fila.</div>


<div style="text-align: justify; font-size: 20px;">Una de las razones por las cuales debe ser un vector fila y no un vector columna es que la regla de la cadena aún es aplicable. Esto se debe a que $\frac{\partial {f}}{\partial \mathbf{x}}$ es un operador lineal que mapea $\Delta{\mathbf{x}}$ en $\Delta f$. Esto puede expresarse utilizando la aproximación de Taylor, como se muestra a continuación:</div>


<span style="font-size:1.4em;">
\begin{equation}
   {f}(\mathbf{x} + \Delta \mathbf{x}) \approx  f(\mathbf{x}) +   \frac{\partial {f}}{\partial \mathbf{x}}\bigg|_{\mathbf{x}} \Delta \mathbf{x}
    \end{equation}
</span>

<div style="text-align: justify; font-size: 20px;">Demostrando que la única forma en la que la aproximación de Taylor tiene sentido es que $\frac{\partial {f}}{\partial \mathbf{x}}$ sea un vector fila.</div>


<div style="text-align: justify; font-size: 20px;"> De igual forma si se tiene la siguiente funcion vectorial:</div>


<span style="font-size:1.4em;">
\begin{equation}
   {g}(\mathbf{y}): \mathbb{R}^{m} \rightarrow \mathbb{R}^{n} 
\end{equation}
</span>

<div style="text-align: justify; font-size: 20px;"> De la cual se puede calcular su derivada parcial de la siguiente forma:</div>


<span style="font-size:1.4em;">
\begin{equation}
   \frac{\partial \mathbf{g}}{\partial \mathbf{y}} \in \mathbb{R}^{n \times m} 
\end{equation}
</span>

<div style="text-align: justify; font-size: 20px;">Esto puede expresarse utilizando la aproximación de Taylor, como se muestra a continuación:</div>


<span style="font-size:1.4em;">
\begin{equation}
   \mathbf{g}(\mathbf{y} + \Delta \mathbf{y}) \approx  \mathbf{g}(\mathbf{y}) +   \frac{\partial \mathbf{g}}{\partial \mathbf{y}}\bigg|_{\mathbf{y}} \Delta \mathbf{y}
    \end{equation}
</span>

<div style="text-align: justify; font-size: 20px;"> Estas definiciones hacen "Chain rule" funciones apropiadamente, esto se demuestra a continuacion:</div>

<span style="font-size:1.4em;">
\begin{equation}
   {f}(\mathbf{g}(\mathbf{y} + \Delta \mathbf{y})) \approx  f(\mathbf{g}(\mathbf{y})) +   \frac{\partial {f}}{\partial \mathbf{x}}\bigg|_{\mathbf{g}(\mathbf{y})} \frac{\partial \mathbf{g}}{\partial \mathbf{y}}\bigg|_{\mathbf{y}}  \Delta \mathbf{y}
    \end{equation}
</span>

<div style="text-align: justify; font-size: 20px;">Por otro lado, por conveniencia podemos definir el gradiente de la siguiente forma:</div>


<span style="font-size:1.4em;">
\begin{equation}
   \nabla f(\mathbf{x}) = (\frac{\partial {f}}{\partial \mathbf{x}})^{T} \in \mathbb{R}^{n \times 1} 
\end{equation}
</span>

<div style="text-align: justify; font-size: 20px;">Finalmente es posible definir el Hessiano de la siguiente forma:</div>


<span style="font-size:1.4em;">
\begin{equation}
   \nabla^{2} f(\mathbf{x}) = \frac{\partial}{\mathbf{x}} \nabla f(\mathbf{x}) = \frac{\partial^2 {f}}{\partial^2 \mathbf{x}}\in \mathbb{R}^{n \times n}
\end{equation}
</span>

<h3 align="left" style="font-size: 20px; color: #667388;"><strong>Root-Finding Algorithm</strong></h3>


<div style="text-align: justify; font-size: 20px;">Este algoritmo se encarga de encontrar las raíces de un vector funcional. Esto puede expresarse de la siguiente forma:</div>


<span style="font-size:1.4em;">
\begin{equation}
   \text{Dada la siguiente funcion}~~~~~~f({\mathbf{x}})~~~~~~\text{encontrar}~~~~~~~~\mathbf{x}^{*}~~~~~~~\text{tal que}~~~~~~f({\mathbf{x}^{*}}) = \mathbf{0}
\end{equation}
</span>

<div class="alert alert-block alert-warning" style="font-size: 16px;">  
  <b>Nota:</b> Un ejemplo práctico son los puntos de equilibrio de un sistema dinámico $\mathbf{f}(\mathbf{x}, \mathbf{u}) = \mathbf{0}$.
</div>


<div style="text-align: justify; font-size: 20px;">Dicho concepto se encuentra relacionado con el "Fixed point", el cual puede ser escrito como:</div>


<span style="font-size:1.4em;">
\begin{equation}
   f({\mathbf{x}^{*}}) = \mathbf{x}^{*}
\end{equation}
</span>

<div class="alert alert-block alert-warning" style="font-size: 16px;">  
  <b>Nota:</b> Un ejemplo práctico son los puntos de equilibrio de un sistema dinámico en tiempo discreto.
</div>
